In [ ]:
pip install presidio-analyzer

  Using cached presidio_analyzer-2.2.355-py3-none-any.whl.metadata (2.9 kB)
  Using cached phonenumbers-8.13.51-py2.py3-none-any.whl.metadata (10 kB)
  Using cached tldextract-5.1.3-py3-none-any.whl.metadata (11 kB)
  Using cached requests_file-2.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached presidio_analyzer-2.2.355-py3-none-any.whl (109 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 7.0 MB/s eta 0:00:00


In [ ]:
pip install presidio-anonymizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 13.2 MB/s eta 0:00:00


In [ ]:
pip install cryptography

In [ ]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig
from cryptography.fernet import Fernet

In [ ]:
# Generate a key for encryption/decryption
key = Fernet.generate_key()
cipher_suite = Fernet(key)

In [ ]:
# Initialize Presidio analyzer and anonymizer
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

In [ ]:
# Custom encrypt function
def encrypt_text(text):
    return cipher_suite.encrypt(text.encode()).decode()

In [ ]:
# Custom decrypt function
def decrypt_text(encrypted_text):
    return cipher_suite.decrypt(encrypted_text.encode()).decode()

In [ ]:
# Function to anonymize text with encryption
def anonymize_text_with_encryption(text):
    # Analyze the text for PII
    analyzer_results = analyzer.analyze(text=text, entities=None, language='en')

    # Prepare operators for detected PII
    operators = {}
    encrypted_mapping = {}  # Store encrypted tokens mapped to original values
    for result in analyzer_results:
        encrypted_value = encrypt_text(result.entity_type)
        operators[result.entity_type] = OperatorConfig("replace", {"new_value": encrypted_value})
        encrypted_mapping[encrypted_value] = result.entity_type

    # Anonymize the text
    anonymized_results = anonymizer.anonymize(
        text=text,
        analyzer_results=analyzer_results,
        operators=operators
    )
    return anonymized_results.text, encrypted_mapping

In [ ]:
# Function to reconstruct text from encrypted anonymized version
def reconstruct_text(anonymized_text, encrypted_mapping):
    for encrypted_value, original_value in encrypted_mapping.items():
        # Replace all occurrences of the encrypted token with the original value
        anonymized_text = anonymized_text.replace(encrypted_value, original_value)
    return anonymized_text

In [ ]:
# Example usage
if __name__ == "__main__":
    text = '''James 'Jimmy' Rodriguez, commonly called Jimmy, resides at 123 Elm Street, Springfield, Illinois, 62701. Born on April 23, 1985, he identifies as a male and is heterosexual. He is of Hispanic origin and belongs to the Catholic faith, regularly attending St. Mary's Church. Jimmy is a union member of the Springfield Workers' Alliance and supports progressive political opinions, frequently sharing his views on local governance. His contact details include a phone number, (217) 555-1234, a fax number, (217) 555-5678, and email addresses, jimmy.rodriguez@gmail.com (personal) and j.rodriguez@workplace.com (professional). His Social Security Number is 123-45-6789.
He holds a U.S. passport (ID: 987654321) and a driver's license (ID: D123456789), and his immigration status is U.S. Citizen. Jimmy's financial details include a checking account number 654321987 at First National Bank, a Visa credit card number 4111-1111-1111-1111 (expiry: 12/26, CVV: 123), and a prepaid card ending in 4321. He owns a certified professional license (ID: CP56789) and a vehicle with the license plate number IL-12345.
Jimmy uses a smartphone with the serial number 9876-5432-1011 and frequently updates his blog at jimmythoughts.com. His home IP address is 192.168.1.1. He has a browsing history that includes visiting springfieldnews.com, and his cookies often reveal product preferences. Jimmy's biometric data includes fingerprints registered with the local police department and a voice profile for his smart assistant.
In addition to his full face photographic image, his medical history includes a diagnosis of Type 2 diabetes (medical record number: MR123456). He is under a health plan with beneficiary ID HP98765. His genetic data revealed a predisposition to cardiovascular issues. Jimmy’s purchase history includes recent acquisitions of fitness equipment and prescription medications.
Professionally, Jimmy works as an IT Manager for Springfield Tech Solutions, where he uses the pseudonym 'TechJimmy' for client interactions. His business profile is maintained on platforms like LinkedIn and includes pseudonymous details. He regularly sends and receives confidential work emails, such as chats about sensitive project information. Lastly, his unique ID code at work is EMP12345'''


In [ ]:
# Anonymize the text
anonymized_text, encrypted_mapping = anonymize_text_with_encryption(text)
print("Anonymized Text:", anonymized_text)



Anonymized Text: gAAAAABnT1dmAWKFBa8BIuibkSE_agF4VKvrUSk9x9cGkOdfdFJOI3JczYTkYfxHIPmLm0i4OXLWFq6DqetRdvFxhfmmE4KqRA==, commonly called gAAAAABnT1dmAWKFBa8BIuibkSE_agF4VKvrUSk9x9cGkOdfdFJOI3JczYTkYfxHIPmLm0i4OXLWFq6DqetRdvFxhfmmE4KqRA==, resides at gAAAAABnT1dmHUH1cmMX7_SnALB9JBtx9YyGTi6GK5cKclcx1piqbXySndqVbqZQfNIGzDIdo6Vosh8C58JzbkbwSFR9QChLWQ==, gAAAAABnT1dmHUH1cmMX7_SnALB9JBtx9YyGTi6GK5cKclcx1piqbXySndqVbqZQfNIGzDIdo6Vosh8C58JzbkbwSFR9QChLWQ==, gAAAAABnT1dmHUH1cmMX7_SnALB9JBtx9YyGTi6GK5cKclcx1piqbXySndqVbqZQfNIGzDIdo6Vosh8C58JzbkbwSFR9QChLWQ==, 62701. Born on gAAAAABnT1dm8dZj2PWb6cHabo1LfmSyCYxrnMnWa9WT907vkleU-xbc2_4nBvHvaVttNQvmNMSWfjjNCuITVsTTiGVRkUj-fQ==, he gAAAAABnT1dmf_BEH60ytXNj4n_ai4CPv5f81BgYnbRmOx5D4S2cTO1DJkTgE2MNDUmFuIAl_G3iW2P5t4e5Hpme73UChOqcmw== as a male and is heterosexual. He is of gAAAAABnT1dmRnbzZJR9-9fEFF1niCDG2auQp6WQKFRKtI-06i-Lszn_ZXOkezfUQs-9vKew9iq45WEPRJJPJOH5xBlSQrw3qg== origin and belongs to the gAAAAABnT1dmRnbzZJR9-9fEFF1niCDG2auQp6WQKFRKtI-06i-Lszn_ZX

In [ ]:

# Reconstruct the text
reconstructed_text = reconstruct_text(anonymized_text, encrypted_mapping)
print("Reconstructed Text:", reconstructed_text)

Reconstructed Text: PERSON, commonly called PERSON, resides at LOCATION, LOCATION, LOCATION, 62701. Born on DATE_TIME, he IN_PAN as a male and is heterosexual. He is of NRP origin and belongs to the NRP faith, regularly attending St. Mary's Church. PERSON is a union member of the Springfield Workers' Alliance and supports progressive political opinions, IN_PAN sharing his views on local IN_PAN. His contact details include a phone number, PHONE_NUMBER, a fax number, PHONE_NUMBER, and email addresses, EMAIL_ADDRESS (personal) and EMAIL_ADDRESS (professional). His Social Security Number is 123-45-6789.
He holds a LOCATION passport (ID: US_PASSPORT) and a driver's license (ID: US_DRIVER_LICENSE), and his immigration status is U.S. Citizen. PERSON's financial details include a checking account number US_BANK_NUMBER at First National Bank, a Visa credit card number CREDIT_CARD (expiry: DATE_TIME, CVV: 123), and a prepaid card ending in DATE_TIME. He owns a certified professional license (ID:

##SCRUBADUB -built on regex

###Entities Detected Natively by scrubadub:

Names

Phone Numbers

Social Security Numbers (SSNs)

IP Addresses

URLs/Blogs

Dates

Credit/Debit Card Numbers

License Plates (limited configurations)

###Entities Not Detected Natively by scrubadub:

PII Related to Personal Attributes:

Full or Partial Addresses,
Gender,
Race/Ethnicity,
Sexual Orientation,
Religious/Philosophical Beliefs,
Union Membership,
Political Opinions,
Identifiers and Financial Data:,
Passport/Government IDs,
Bank Account Numbers,
Prepaid/Gift Card Numbers,
Device Identifiers,
Biometric Data,

####Health and Medical Data:

Medical Records,
Health Plan IDs,
Genetic Data,
Web and Digital Footprint:
Cookies/Browsing History,
Personalized User Profiles,
Other:
Employment Data,
Purchase History,
Unique Identifying Numbers

In [ ]:
pip install scrubadub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.5/636.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.4 MB/s eta 0:00:00
  Attempting uninstall: textblob
    Found existing installation: textblob 0.17.1
    Uninstalling textblob-0.17.1:
      Successfully uninstalled textblob-0.17.1


In [ ]:
import scrubadub

# Sample text containing various PII
text = """
James 'Jimmy' Rodriguez, commonly called Jimmy, resides at 123 Elm Street, Springfield, Illinois, 62701.
Born on April 23, 1985, he identifies as a male and is heterosexual. He is of Hispanic origin and belongs to the Catholic faith, regularly attending St. Mary's Church.
Jimmy is a union member of the Springfield Workers' Alliance and supports progressive political opinions, frequently sharing his views on local governance.
His contact details include a phone number, (217) 555-1234, a fax number, (217) 555-5678, and email addresses, jimmy.rodriguez@gmail.com (personal) and j.rodriguez@workplace.com (professional).
His Social Security Number is 123-45-6789. He holds a U.S. passport (ID: 987654321) and a driver's license (ID: D123456789), and his immigration status is U.S. Citizen.
Jimmy's financial details include a checking account number 654321987 at First National Bank, a Visa credit card number 4111-1111-1111-1111 (expiry: 12/26, CVV: 123),
and a prepaid card ending in 4321. He owns a certified professional license (ID: CP56789) and a vehicle with the license plate number IL-12345.
Jimmy uses a smartphone with the serial number 9876-5432-1011 and frequently updates his blog at jimmythoughts.com.
His home IP address is 192.168.1.1. He has a browsing history that includes visiting springfieldnews.com, and his cookies often reveal product preferences.
Jimmy's biometric data includes fingerprints registered with the local police department and a voice profile for his smart assistant.
In addition to his full face photographic image, his medical history includes a diagnosis of Type 2 diabetes (medical record number: MR123456).
He is under a health plan with beneficiary ID HP98765. His genetic data revealed a predisposition to cardiovascular issues.
Jimmy’s purchase history includes recent acquisitions of fitness equipment and prescription medications.
Professionally, Jimmy works as an IT Manager for Springfield Tech Solutions, where he uses the pseudonym 'TechJimmy' for client interactions.
His business profile is maintained on platforms like LinkedIn and includes pseudonymous details.
He regularly sends and receives confidential work emails, such as chats about sensitive project information.
Lastly, his unique ID code at work is EMP12345.
"""

# Use scrubadub to detect PII
cleaned_text = scrubadub.clean(text)

# Print anonymized text
print("Anonymized Text:")
print(cleaned_text)


Anonymized Text:

James 'Jimmy' Rodriguez, commonly called Jimmy, resides at 123 Elm Street, Springfield, Illinois, 62701. 
Born on April 23, 1985, he identifies as a male and is heterosexual. He is of Hispanic origin and belongs to the Catholic faith, regularly attending St. Mary's Church.
Jimmy is a union member of the Springfield Workers' Alliance and supports progressive political opinions, frequently sharing his views on local governance. 
His contact details include a phone number, {{PHONE}}, a fax number, {{PHONE}}, and email addresses, {{EMAIL}} (personal) and {{EMAIL}} (professional). 
His Social Security Number is {{SOCIAL_SECURITY_NUMBER}}. He holds a U.S. passport (ID: 987654321) and a driver's license (ID: D123456789), and his immigration status is U.S. Citizen. 
Jimmy's financial details include a checking account number 654321987 at First National Bank, a Visa credit card number 4111-1111-1111-1111 (expiry: 12/26, CVV: 123), 
and a prepaid card ending in 4321. He owns a 

##anonLLM - built on SpaCY (NER), Regex, Presidio

###Integration with APIs:

####Prepares the text for submission to APIs like GPT ( OpenAI, Cohere) while ensuring sensitive information is masked.

####Focuses on preprocessing steps to safeguard data privacy

In [ ]:
pip install anonLLM

In [ ]:
from anonLLM.anonymizer import Anonymizer

In [ ]:
anonymizer = Anonymizer()

In [ ]:
# Input text
#input_text = "My name is Alice Johnson, email: alice.johnson@example.com, phone: +1 234-567-8910."
input_text = '''James 'Jimmy' Rodriguez, commonly called Jimmy, resides at 123 Elm Street.'''


In [ ]:
# Ensure input_text is treated as a single string
preprocessed_input_text = str(input_text)

# Call the anonymizer with the preprocessed input
anonymized_text, _ = anonymizer.anonymize_data(preprocessed_input_text)

# Print only the anonymized text
print("Anonymized Text:")
print(anonymized_text)



Anonymized Text:
Cynthia Hensley, commonly called Jimmy, resides at 123 Elm Street.


In [ ]:
!git clone https://github.com/redhuntlabs/Octopii.git


Cloning into 'Octopii'...
remote: Enumerating objects: 306, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 306 (delta 138), reused 177 (delta 102), pack-reused 74 (from 1)
Receiving objects: 100% (306/306), 4.42 MiB | 7.08 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [ ]:
%cd Octopii


/content/Octopii/Octopii/Octopii


In [ ]:
!pip install -r requirements.txt


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
!pip install textract==1.6.3


In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root

True

In [ ]:
!sudo apt-get install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!python octopii.py /content/Octopii/dummy-pii


Scanning '/content/Octopii/dummy-pii'
{
    "file_path": "/content/Octopii/dummy-pii/dummy-hong-kong-resident-id.png",
    "pii_class": "Permanent Identity Card",
    "score": 20,
    "country_of_origin": "Hong Kong",
    "faces": 2,
    "identifiers": [
        "2867 3057 2532"
    ],
    "emails": [],
    "phone_numbers": [],
    "addresses": [
        "HONG"
    ]
}

Creating new file named 'output.json' and writing to it.

Output saved in output.json
{
    "file_path": "/content/Octopii/dummy-pii/dummy-drivers-license-maharashtra.jpg",
    "pii_class": "Maharashtra Driver's License",
    "score": 28,
    "country_of_origin": "India",
    "faces": 0,
    "identifiers": [
        "fnpressio",
        "MAHARASHT",
        "Irerercnn",
        "GAIGANWAD",
        "HARASHTRA",
        "Impressio",
        "Signature",
        "BAIGANWAD",
        "FOLLOWING",
        "THORISATI",
        "THROUGHOU"
    ],
    "emails": [],
    "phone_numbers": [
        "2633 325381"
    ],
    "addre

In [ ]:
!python octopii.py /content/test.txt


Scanning '/content/test.txt'
{
    "file_path": "/content/test.txt",
    "pii_class": "Tawuniya Health Insurance",
    "score": 9,
    "country_of_origin": "Saudi Arabia",
    "faces": 0,
    "identifiers": [
        "4111-1111-1111",
        "9876-5432-1011"
    ],
    "emails": [
        "j.rodriguez@workplace.com",
        "jimmy.rodriguez@gmail.com"
    ],
    "phone_numbers": [
        "(217) 555-5678",
        "(217) 555-1234"
    ],
    "addresses": [
        "Illinois",
        "Elm Street",
        "Hispanic",
        "U.S.",
        "Springfield"
    ]
}

Output saved in output.json


In [ ]:
pip install datafog

In [ ]:
datafog scan-text "Tim Cook is the CEO of Apple and is based out of Cupertino, California"

SyntaxError: invalid syntax (<ipython-input-3-5e45b44e020b>, line 1)